In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData, UVCal, UVFlag, utils
from glob import glob
from astropy import units as u
from astropy import constants as c
from astropy.time import Time
from astropy.coordinates import get_body, SkyCoord, AltAz, EarthLocation, Angle
from copy import deepcopy

In [2]:
rawpath = '/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/'
calpath = '/lustre/aoc/projects/hera/agavidia/TimeVariability/CalFitsFix/'
outpath = '/lustre/aoc/projects/hera/agavidia/TimeVariability/CalFitsFlag/'

In [3]:
JD = '2457548'
files = glob(calpath+JD+'/'+ 'zen.'+JD+'**'+'fixed.uvcRP.calfits')
rawfiles = glob(rawpath+JD+'/'+'zen'+JD+'**'+'uvcRP.calfits')

In [4]:
uvc = UVCal()
uvd = UVData()

In [5]:
%%time 

for file in files:
    uvc.read_calfits(file)
    for rawfile in rawfiles:
        uvd.read(rawfiles)
        uvd_calibrated = uvd
        uvd_calibrated.flag_array = np.zeros_like(uvd.flag_array, dtype='bool')
        utils.uvcalibrate(uvd_calibrated, uvc, inplace=True, prop_flags=True)
        uvf_or = UVFlag(uvc, mode='flag', copy_flags=True)
        uvd_cal_flag = utils.apply_uvflag(uvd_calibrated, uvf_or, inplace=False, unflag_first=False)
        filename = file.split('/')[-1]
        tmp = filename.split('.')
        juldate = tmp[1]+'.'+tmp[2]
        uvc.write_calfits(outpath+JD+'/zen.'+juldate+'_flag.or.uvcRP.calfits')

CPU times: user 4.29 s, sys: 11 s, total: 15.3 s
Wall time: 24.4 s


In [ ]:
#ls /lustre/aoc/projects/hera/agavidia/TimeVariability/CalFitsFix/2457548

In [ ]:
ls /lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/